In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import os
from collections import Counter
import numpy as np

In [24]:
DATA_DIR = '/content/drive/MyDrive/Terraria/dataset'
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
NUM_EPOCHS = 15
LEARNING_RATE = 0.001
NUM_CLASSES = 11

In [25]:
CLASS_NAMES = [
    "ak", "pistol", "rifle", "helmet", "pounch",
    "shutgun", "vest", "M series", "mashinegun", "backpack",
    "other"
]

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [27]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [28]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = CLASS_NAMES
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = self._make_dataset()

    def _make_dataset(self):
        samples = []
        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue
            for filename in os.listdir(class_dir):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    path = os.path.join(class_dir, filename)
                    item = (path, self.class_to_idx[class_name])
                    samples.append(item)
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [29]:
full_dataset = CustomDataset(root_dir=DATA_DIR, transform=data_transforms['train'])

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

val_dataset.dataset.transform = data_transforms['val']

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [30]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 167MB/s]


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [32]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS):
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_model.pth')

        print()

    print(f'Best val Acc: {best_acc:.4f}')
    return model

In [33]:
model = train_model(model, criterion, optimizer, scheduler)

Epoch 0/14
----------
train Loss: 0.6862 Acc: 0.7936
val Loss: 2.4867 Acc: 0.6288

Epoch 1/14
----------
train Loss: 0.2689 Acc: 0.9129
val Loss: 0.6508 Acc: 0.8636

Epoch 2/14
----------
train Loss: 0.1358 Acc: 0.9583
val Loss: 1.8496 Acc: 0.6894

Epoch 3/14
----------
train Loss: 0.0832 Acc: 0.9792
val Loss: 0.2924 Acc: 0.9470

Epoch 4/14
----------
train Loss: 0.0506 Acc: 0.9811
val Loss: 0.3502 Acc: 0.8864

Epoch 5/14
----------
train Loss: 0.0441 Acc: 0.9924
val Loss: 0.3284 Acc: 0.9091

Epoch 6/14
----------
train Loss: 0.0905 Acc: 0.9754
val Loss: 0.8993 Acc: 0.8106

Epoch 7/14
----------
train Loss: 0.0293 Acc: 0.9905
val Loss: 0.3051 Acc: 0.9167

Epoch 8/14
----------
train Loss: 0.0085 Acc: 0.9981
val Loss: 0.2210 Acc: 0.9394

Epoch 9/14
----------
train Loss: 0.0096 Acc: 1.0000
val Loss: 0.2136 Acc: 0.9318

Epoch 10/14
----------
train Loss: 0.0050 Acc: 1.0000
val Loss: 0.2103 Acc: 0.9394

Epoch 11/14
----------
train Loss: 0.0044 Acc: 1.0000
val Loss: 0.2150 Acc: 0.9470

Ep

In [34]:
import requests
from io import BytesIO
import torch.nn.functional as F

def save_model(model, path='weapon_classifier.pth'):
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_names': CLASS_NAMES
    }, path)
    print(f"Model saved to {path}")


In [36]:
save_model(model)

Model saved to weapon_classifier.pth
